In [3]:
!pip install python-dotenv
!pip install --upgrade --quiet langchain-core
!pip install --upgrade --quiet langchain
!pip install --upgrade --quiet langchain-community
!pip install --upgrade --quiet langchain-together

In [5]:
import os
from dotenv import load_dotenv
from langchain_together import ChatTogether

load_dotenv(override=True)

llm = ChatTogether(api_key=os.getenv("TOGETHER_API_KEY"), temperature=0.0, model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo")

messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    (
        "human",
        "I love apples",
    )
]

ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content="Je t'aime les pommes.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 55, 'total_tokens': 64}, 'model_name': 'meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo', 'system_fingerprint': None, 'finish_reason': 'eos', 'logprobs': None}, id='run-ac6324f3-df58-46ff-8c62-0eed0ad8ddc4-0', usage_metadata={'input_tokens': 55, 'output_tokens': 9, 'total_tokens': 64, 'input_token_details': {}, 'output_token_details': {}})

In [6]:
ai_msg.content

"Je t'aime les pommes."

Using chains in Langchain

In [7]:
from langchain.chains.router import MultiPromptChain # for creating a chain with multiple prompts
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser # for routing logic in LLM chains
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain import LLMChain

In [8]:
# Define templates for handling different types of queries

flight_status_template = """You are a helpful airline customer service representative.
Answer the following query about flight status. Provide accurate, concise, and friendly information.
Here is the query:
{input}"""

baggage_inquiry_template = """You are a knowledgeable airline customer service representative.
Provide information regarding baggage policies, lost baggage, or baggage fees.
Here is the query:
{input}"""

ticket_booking_template = """You are an airline booking agent. Assist
the customer in booking, modifying, or canceling a flight.
Here is the request:
{input}"""

general_inquiry_template = """You are a customer service representative at an airline.
Answer the following general inquiry about the airline, its services, or policies.
Here is the question:
{input}"""

In [9]:
# Define prompt info for different routes

prompt_infos = [
    {
        "name": "flight_status",
        "description": "Handles inquiries related to flight status.",
        "prompt_template": flight_status_template
    },
    {
        "name": "baggage_inquiry",
        "description": "Good for responding to questions about baggage, including fees and lost items.",
        "prompt_template": baggage_inquiry_template
    },
    {
        "name": "ticket_booking",
        "description": "Handles ticket booking, modification or cancellation requests.",
        "prompt_template": ticket_booking_template
    },
    {
        "name": "general_inquiry",
        "description": "Good for answering general inquiries about the airline or its services or handling feedbacks.",
        "prompt_template": general_inquiry_template
    }
]

In [11]:
# Create a chain destination for each feedback type in dictionary (destination_chains)
# Each chain is created by combining the ChatPromptTemplate with LLMChain

destination_chains = {}

for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

# Create destinations string for router template
destinations = [f"{p["name"]}: {p["description"]}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

Router chains

In [15]:
# Create a fallback hain for queries that do not fit specific category
default_prompt = ChatPromptTemplate.from_template("reply to this {input} gracefully")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

# Define router template
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input, select the prompt best suited for the input. 
You will be given the names of the available prompts and a description of what the prompt is best suited for.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ original input
}}}}
<< CANDIDATE PROMPTS >> {destinations}

<< INPUT >> {{input}}

<< OUTPUT (remember to include the ```json)>>
"""

<>:6: SyntaxWarning: invalid escape sequence '\ '
<>:6: SyntaxWarning: invalid escape sequence '\ '
/var/folders/_6/sw6vhpc10411wn4lc23xd4t80000gn/T/ipykernel_50193/3373844381.py:6: SyntaxWarning: invalid escape sequence '\ '
  MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input, select the prompt best suited for the input.


In [16]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(template=router_template, input_variables=["input"], output_parser=RouterOutputParser())
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

Combining chains

In [17]:
chain = MultiPromptChain(router_chain=router_chain, destination_chains=destination_chains, default_chain=default_chain, verbose=True)

/var/folders/_6/sw6vhpc10411wn4lc23xd4t80000gn/T/ipykernel_50193/723251918.py:1: LangChainDeprecationWarning: Use RunnableLambda to select from multiple prompt templates. See example in API reference: https://api.python.langchain.com/en/latest/chains/langchain.chains.router.multi_prompt.MultiPromptChain.html
  chain = MultiPromptChain(router_chain=router_chain, destination_chains=destination_chains, default_chain=default_chain, verbose=True)


In [18]:
chain.invoke("What is the current status of flight AA123?")



> Entering new MultiPromptChain chain...
flight_status: {'input': 'What is the current status of flight AA123?'}
> Finished chain.


{'input': 'What is the current status of flight AA123?',
 'text': "I'd be happy to help you with the status of flight AA123. \n\nTo check the current status, I'll need to look up the flight information. Can you please tell me the departure and arrival cities for flight AA123, as well as the date of travel? This will help me provide you with the most accurate information."}

In [19]:
chain.invoke("Can you tell me about your airline's safety protocols?")



> Entering new MultiPromptChain chain...
general_inquiry: {'input': "Can you tell me about your airline's safety protocols?"}
> Finished chain.


{'input': "Can you tell me about your airline's safety protocols?",
 'text': "At [Airline Name], the safety of our passengers and crew is our top priority. We take every precaution to ensure that our flights are operated in a safe and secure environment.\n\nOur airline adheres to the highest safety standards set by the International Air Transport Association (IATA), the Federal Aviation Administration (FAA), and the European Aviation Safety Agency (EASA). We have a robust safety management system in place, which includes regular maintenance checks, crew training, and emergency preparedness procedures.\n\nHere are some of the key safety protocols we have in place:\n\n1. **Regular Aircraft Maintenance**: Our aircraft undergo regular maintenance checks, including daily pre-flight inspections, to ensure that every plane is airworthy before each flight.\n2. **Crew Training**: Our pilots and cabin crew undergo rigorous training programs to prepare them for emergency situations, including eva

In [20]:
chain.invoke("How can I upgrade my flight ticket that is already booked?")



> Entering new MultiPromptChain chain...
ticket_booking: {'input': 'How can I upgrade my flight ticket that is already booked?'}
> Finished chain.


{'input': 'How can I upgrade my flight ticket that is already booked?',
 'text': "I'd be happy to help you upgrade your flight ticket. \n\nTo upgrade your flight ticket, I'll need to check a few things first. Can you please provide me with the following information:\n\n1. Your booking reference number or ticket number?\n2. Your name as it appears on the ticket?\n3. The flight number and date of travel?\n4. The class of service you're currently booked in (economy, premium economy, business, or first class)?\n5. The class of service you'd like to upgrade to?\n\nAdditionally, please note that upgrades are subject to availability and may incur an additional fee. I'll also need to check if there are any upgrade options available on your flight.\n\nOnce I have this information, I can check for upgrade options and let you know what's available."}

In [22]:
chain.invoke("I could not find my baggage after arrival. What should I do?")



> Entering new MultiPromptChain chain...
baggage_inquiry: {'input': 'I could not find my baggage after arrival. What should I do?'}
> Finished chain.


{'input': 'I could not find my baggage after arrival. What should I do?',
 'text': "I'm so sorry to hear that you're experiencing issues with your baggage. If you couldn't find your baggage after arrival, here's a step-by-step guide to help you resolve the issue:\n\n1. **Report the issue immediately**: As soon as you realize your baggage is missing, please report it to our airline's baggage claim area or to one of our staff members at the airport. You can also report it to our customer service desk or contact us through our mobile app or website.\n2. **Provide your baggage claim tag**: When reporting the issue, please provide your baggage claim tag, which is usually attached to your baggage. This will help us identify your baggage and locate it.\n3. **Fill out a Property Irregularity Report (PIR)**: Our staff will guide you through the process of filling out a PIR, which is a report that helps us track and locate your baggage. This report will also help us to investigate the cause of t